In [ ]:
!pip install pycryptodome transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import base64
import json

# Generate AES Encryption Key
def generate_key():
    return get_random_bytes(32)

# Save the Key
def save_key(key, key_file):
    with open(key_file, "wb") as f:
        f.write(key)
    print(f"Key saved to: {key_file}")

# Encryption Function for Dataset
def encrypt_dataset(dataset, key, output_file):
    dataset_dict = {split: dataset[split].to_dict() for split in dataset.keys()}
    dataset_str = json.dumps(dataset_dict)

    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(dataset_str.encode())

    encrypted_data = {
        "ciphertext": base64.b64encode(ciphertext).decode(),
        "nonce": base64.b64encode(cipher.nonce).decode(),
        "tag": base64.b64encode(tag).decode(),
    }

    with open(output_file, "w") as f:
        json.dump(encrypted_data, f)
    print(f"Encrypted dataset saved to: {output_file}")

# Generate and Save Key
key = generate_key()
save_key(key, "encryption_key.key")

# Encrypt Dataset
from datasets import load_dataset
dataset = load_dataset("holistic-ai/EMGSD")
encrypt_dataset(dataset, key, "encrypted_emgsd.json")


Key saved to: encryption_key.key


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/8.91M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45760 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11441 [00:00<?, ? examples/s]

Encrypted dataset saved to: encrypted_emgsd.json


In [ ]:
from Crypto.Cipher import AES
import base64
import json

# Load the Key
def load_key(key_file):
    with open(key_file, "rb") as f:
        return f.read()

# Decryption Function for Dataset
def decrypt_dataset(encrypted_file, key):
    with open(encrypted_file, "r") as f:
        encrypted_data = json.load(f)

    ciphertext = base64.b64decode(encrypted_data["ciphertext"])
    nonce = base64.b64decode(encrypted_data["nonce"])
    tag = base64.b64decode(encrypted_data["tag"])

    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    dataset_str = cipher.decrypt_and_verify(ciphertext, tag)

    return json.loads(dataset_str)

# Load Key and Decrypt Dataset
key = load_key("encryption_key.key")
decrypted_dataset = decrypt_dataset("encrypted_emgsd.json", key)
# Save Decrypted Dataset to a JSON File
with open("decrypted_emgsd.json", "w") as f:
    json.dump(decrypted_dataset, f, indent=4)
print("Decrypted dataset saved to: decrypted_emgsd.json")

print("Decryption successful!")
# print(decrypted_dataset["train"][:2])


Decrypted dataset saved to: decrypted_emgsd.json
Decryption successful!


In [5]:

!pip install transformers torch datasets shap pandas scikit-learn accelerate matplotlib codecarbon==2.4.2 lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.9/494.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.2 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=e08e8f2acd729b36c7b5ddc71dc487b270f5b88f929e7a1a31963a2dedc900f1
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [6]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive
